In [1]:
# Khai báo các thư viện cần thiết
from new_import import *

# Khai báo đường dẫn đến kết quả phân loại và kết quả thực tế của địa phương
KD_path = "ThuanHoa/KhoanhDat/ThuanHoa_TKDD2022.shp"
KetQuaPhanLoaiDat = "KetQuaPhanLoaiDat_15-09.tif"

In [4]:
gdf = gpd.read_file(KD_path, crs="EPSG:9209")
gdf.head(10)

,id,thuaid,xaid,shbando,shthua,dientich,dientichpl,maloaidat,khloaidat,diadanh,dtsd,tenchu,diachi,mdsd2003,kh2003,mscolor,shthuatam,dtthocu,geometry
0,0,7620220493,31576,202,493,5058.2,0.0,50,None,None,1,SKC,TKT,33,SKC,None,None,0,"POLYGON ((540179.700 1070550.084, 540209.964 1..."
1,1,7620220456,31576,202,456,407.6,0.0,53,None,None,1,ONT,GDC,26,ONT,None,None,0,"POLYGON ((540278.458 1070634.382, 540300.244 1..."
2,2,7620220465,31576,202,465,4537.2,0.0,-1,None,None,1,ONT(1782.60);CLN(2754.70),GDC,-1,None,None,None,0,"POLYGON ((540212.264 1070577.653, 540278.458 1..."
3,3,7620220451,31576,202,451,397.9,0.0,53,None,None,1,ONT,GDC,26,ONT,None,None,0,"POLYGON ((540315.403 1070666.044, 540325.424 1..."
4,4,7620220494,31576,202,494,3016.7,0.0,6,None,None,1,LUC,GDC,1,LUC,None,None,0,"POLYGON ((540301.202 1070519.021, 540315.563 1..."
5,5,7620220885,31576,202,885,906.4,0.0,-1,None,None,1,ONT(297.10);CLN(609.30),GDC,-1,None,None,None,0,"POLYGON ((540325.424 1070674.633, 540340.356 1..."
6,6,7620220476,31576,202,476,2501.7,0.0,13,None,None,1,BHK,GDC,5,BHK,None,None,0,"POLYGON ((540297.974 1070574.813, 540300.514 1..."
7,7,7620220446,31576,202,446,521.7,0.0,42,None,None,1,DGT,UBQ,36,DGT,None,None,0,"POLYGON ((540340.356 1070687.429, 540344.227 1..."
8,8,7620220906,31576,202,906,3148.1,0.0,18,None,None,1,CLN,GDC,65,CLN,None,None,0,"POLYGON ((540327.344 1070601.823, 540327.234 1..."
9,9,7620220417,31576,202,417,4187.5,0.0,18,None,None,1,CLN,GDC,65,CLN,None,None,0,"POLYGON ((540344.227 1070690.747, 540416.194 1..."


In [5]:
# khai báo các loại đất ứng với các mã đất phân loại được
CODE_MAP = {
    "BHK": 2,
    "CLN": 3,
    "DGD": 6,
    "DGT": 6,
    "DNL": 6,
    "DRA": 6,
    "DSH": 6,
    "DTL": 5,
    "DTS": 6,
    "DYT": 6,
    "LUC": 1,
    "NKH": 3,
    "NTD": 6,
    "NTS": 4,
    "ONT": 6,
    "SKC": 6,
    "SKX": 6,
    "SON": 5,
    "TMD": 6,
    "TON": 6,
    "TSC": 6,
}

# Khai báo các nhãn phân loại đất ứng với 3 loại đất chính
HT_MAP = {
    "NN": {"name": "Đất Nông Nghiệp", "data": [1, 2, 3, 4]},
    "PNN": {"name": "Đất Phi Nông Nghiệp", "data": [6]},
    "TQ": {"name": "Đất Thổ Quả", "data": [15]},
}

In [6]:
# Tiến hành chồng lắp
result = compare(KD_path, KetQuaPhanLoaiDat, CODE_MAP, HT_MAP)

process NN
process PNN
process TQ


In [7]:
# cấu hình màu cho các loại sử dụng đất
colors = [
    "#abcee9",
    "#ffffc0",
    "#c4ff9e",
    "#ffd6a8",
    "#93ddda",
    "#1aeef7",
    "#ffa7f2",
    "#33ee33",
]
labels = ["Lúa tôm", "Lúa", "CHN", "CLN", "TS", "Sông", "Đất xây dựng", "Rừng"]

In [8]:
# Lưu kết quả
save_result(result, HT_MAP)

save ThuanHoa/KetQua/NN.tif
save ThuanHoa/KetQua/PNN.tif
save ThuanHoa/KetQua/TQ.tif


In [6]:
# hiển thị kết quả
xx = []

for k, v in result.items():
    rs = merge_arrays(v, nodata=np.nan)
    xx.append(rs.squeeze(drop=True))
xx = xr.concat(xx, pd.Index([HT_MAP[x]["name"] for x in HT_MAP], name="name"))

colorval = list(range(len(colors)))
options = {
    "cmap": colors,
    "clim": (0, 8),
    "aspect": "equal",
    "height": 400,
    "colorbar_opts": {
        "major_label_overrides": dict(zip(colorval, labels)),
        "major_label_text_align": "left",
        "ticker": FixedTicker(ticks=colorval),
    },
}

xx.hvplot(
    groupby="name",
    rasterize=True,  # Use Datashader, particularly useful for dask arrays
    aggregator=reductions.mode(),  # Datashader selects mode value, requires 'hv.Image'
).options(opts.Image(**options))

:DynamicMap   [name]
   :Image   [y,x]   (value)